<a href="https://colab.research.google.com/github/SoudeepGhoshal/ColabFiltering/blob/main/PIP_CollaborativeFiltering_MovieLens1M_KNNImputed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import pairwise_distances
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
cols_u = ['user_id', 'sex', 'age', 'occupation', 'zip_code']
users=pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-1m/users.dat', sep='::', names=cols_u, encoding='latin-1', parse_dates=True)

ratings_u = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings=pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-1m/ratings.dat', sep='::', names=ratings_u, encoding='latin-1')

movies_u = ['movie_id' , 'title', 'genre']
movies = pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-1m/movies.dat', sep='::', names=movies_u, encoding='latin-1')

movie_ratings=pd.merge(movies,ratings)
movie_df=pd.merge(movie_ratings,users)

movie_df.head(500)

<ipython-input-38-3433c4bfd01f>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users=pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-1m/users.dat', sep='::', names=cols_u, encoding='latin-1', parse_dates=True)
<ipython-input-38-3433c4bfd01f>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings=pd.read_csv('/content/drive/MyDrive/IOT_Lab/ml-1m/ratings.dat', sep='::', names=ratings_u, encoding='latin-1')
<ipython-input-38-3433c4bfd01f>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interp

,movie_id,title,genre,user_id,rating,timestamp,sex,age,occupation,zip_code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
495,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,10,5,979167660,F,35,1,95370
496,1198,Raiders of the Lost Ark (1981),Action|Adventure,10,5,978225630,F,35,1,95370
497,1200,Aliens (1986),Action|Sci-Fi|Thriller|War,10,5,979168160,F,35,1,95370
498,1201,"Good, The Bad and The Ugly, The (1966)",Action|Western,10,2,978225853,F,35,1,95370


In [39]:
#only for the normalization
min_rating = ratings['rating'].min()
max_rating = ratings['rating'].max()

ratings['rating'] = (ratings['rating'] - min_rating) / (max_rating - min_rating)

ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,1.00,978300760
1,1,661,0.50,978302109
2,1,914,0.50,978301968
3,1,3408,0.75,978300275
4,1,2355,1.00,978824291


In [40]:
movie_df.drop(movie_df.columns[[2,5]], axis=1, inplace=True)
ratings.drop("timestamp", inplace=True, axis=1)
movies.drop(movies.columns[[2]], inplace=True, axis=1)
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   movie_id    1000209 non-null  int64 
 1   title       1000209 non-null  object
 2   user_id     1000209 non-null  int64 
 3   rating      1000209 non-null  int64 
 4   sex         1000209 non-null  object
 5   age         1000209 non-null  int64 
 6   occupation  1000209 non-null  int64 
 7   zip_code    1000209 non-null  object
dtypes: int64(5), object(3)
memory usage: 68.7+ MB


Run if not using Imputation on ratings

In [ ]:
user_input_percent = 0.3
subset_percent = 0.7

num_ratings = len(ratings)
num_user_input = int(num_ratings * user_input_percent)
num_subset = int(num_ratings * subset_percent)

user_input_ratings = ratings.sample(n=num_user_input, random_state=42)
subset_ratings = ratings.drop(user_input_ratings.index)
user_input_ratings

,user_id,movie_id,rating
895536,5412,2683,0.25
899739,5440,904,1.00
55687,368,3717,0.75
63727,425,1721,0.75
822011,4942,3697,0.00
...,...,...,...
85492,558,923,1.00
586763,3582,329,0.75
589709,3597,3175,0.50
548798,3389,2987,0.75


Run for Imputation

In [41]:
userItemMatrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating')
userItemMatrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,0.25,NaN,0.50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
null_values = userItemMatrix.isnull().sum().sum()
print(f"Total number of null values: {null_values}")

Total number of null values: 21384031


Simple Imputation technique applied here

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Create a copy of the userItemMatrix
userItemMatrix_imputed = userItemMatrix.copy()

# Calculate the total number of null values in the table
total_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the threshold for the number of null values to be imputed
null_threshold = int(total_null_values * 0.60)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')

# Iterate over each row
for index, row in userItemMatrix_imputed.iterrows():
    # Calculate the number of null values in each row
    row_null_values = row.isnull().sum()

    # If the number of null values in the row is less than the threshold, skip imputation for this row
    if row_null_values == 0 or row_null_values > null_threshold:
        continue

    # Create a mask for 60% of the null values in this row
    row_mask = np.random.choice([True, False], len(row), p=[0.6, 0.4])

    # Apply imputation for the selected 60% of null values in this row
    row_values = row.values
    row_values[row_mask] = imputer.fit_transform(row_values[row_mask].reshape(-1, 1)).ravel()

    # Update the values in the DataFrame for this row
    userItemMatrix_imputed.loc[index] = row_values

# Calculate the number of null values after imputation
remaining_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the number of null values that have been imputed
imputed_values = total_null_values - remaining_null_values

# Calculate the percentage of null values imputed
percent_imputed = (imputed_values / total_null_values) * 100

print(f"Percentage of null values imputed: {percent_imputed:.2f}%")
print(f"Remaining null values: {remaining_null_values}")

Percentage of null values imputed: 59.99%
Remaining null values: 8556016


KNN Imputation Technique applied here

In [43]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

# Create a copy of the userItemMatrix
userItemMatrix_imputed = userItemMatrix.copy()

# Calculate the total number of null values in the table
total_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the threshold for the number of null values to be imputed
null_threshold = int(total_null_values * 0.60)

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Perform kNN imputation on the DataFrame
userItemMatrix_imputed = pd.DataFrame(knn_imputer.fit_transform(userItemMatrix_imputed))

# Create a random mask to identify positions for replacement
mask = np.random.choice([True, False], size=userItemMatrix_imputed.shape, p=[0.4, 0.6])

# Update the corresponding values in the DataFrame with np.nan
userItemMatrix_imputed[mask] = np.nan

# Display the DataFrame after imputation
userItemMatrix_imputed

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,1.00,0.65,NaN,NaN,NaN,NaN,0.75,NaN,0.60,NaN,...,0.35,0.35,0.20,NaN,0.50,0.70,0.95,NaN,NaN,0.70
1,NaN,NaN,NaN,NaN,NaN,0.60,0.65,0.50,0.55,0.75,...,0.40,0.40,0.15,NaN,0.60,0.90,NaN,0.55,0.75,0.70
2,0.90,NaN,NaN,NaN,0.45,NaN,0.80,0.55,0.70,NaN,...,0.40,NaN,0.25,0.2,0.35,NaN,0.75,NaN,NaN,NaN
3,NaN,NaN,0.55,0.55,0.20,0.85,0.55,0.50,NaN,0.55,...,0.50,0.30,NaN,0.2,0.45,NaN,NaN,NaN,0.60,0.80
4,NaN,0.35,0.35,NaN,0.30,0.25,0.90,0.45,NaN,0.40,...,0.50,NaN,NaN,0.5,0.35,0.50,0.95,0.60,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.95,0.25,0.40,0.25,0.30,NaN,0.45,NaN,NaN,NaN,...,0.65,NaN,NaN,0.4,NaN,0.85,0.90,0.55,0.80,NaN
6036,0.80,0.55,0.65,0.55,0.55,0.70,NaN,0.45,0.30,0.70,...,NaN,0.45,0.05,0.3,NaN,NaN,0.95,0.45,NaN,0.70
6037,NaN,0.55,0.55,NaN,0.65,NaN,NaN,0.50,NaN,NaN,...,0.30,0.30,0.30,NaN,0.65,0.60,NaN,0.50,0.65,NaN
6038,NaN,0.55,0.40,NaN,NaN,0.65,NaN,NaN,NaN,NaN,...,0.35,0.35,0.10,0.4,NaN,0.70,NaN,NaN,NaN,0.90


In [44]:
# Calculate the total number of values in the DataFrame
total_values = userItemMatrix_imputed.size

# Calculate the number of null values after imputation
remaining_null_values = userItemMatrix_imputed.isnull().sum().sum()

# Calculate the number of null values that have been imputed
imputed_values = total_null_values - remaining_null_values

# Calculate the percentage of null values imputed
percent_imputed = (imputed_values / total_null_values) * 100

print(f"Percentage of null values imputed: {percent_imputed:.2f}%")
print(f"Remaining null values: {remaining_null_values}")

Percentage of null values imputed: 58.13%
Remaining null values: 8954531


In [45]:
#cpmare table sizes for any anomaly
print(userItemMatrix_imputed.shape[1])
print(userItemMatrix.shape[1])

3706
3706


In [46]:
print(len(ratings))

1000209


In [47]:
ratings_series = userItemMatrix_imputed.stack()
ratings_imputed = ratings_series.reset_index()
ratings_imputed.columns = ['user_id', 'movie_id', 'rating']

In [48]:
print(len(ratings_imputed))

13429709


In [49]:
#Testing for new values
userItemMatrix_imputed

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,1.00,0.65,NaN,NaN,NaN,NaN,0.75,NaN,0.60,NaN,...,0.35,0.35,0.20,NaN,0.50,0.70,0.95,NaN,NaN,0.70
1,NaN,NaN,NaN,NaN,NaN,0.60,0.65,0.50,0.55,0.75,...,0.40,0.40,0.15,NaN,0.60,0.90,NaN,0.55,0.75,0.70
2,0.90,NaN,NaN,NaN,0.45,NaN,0.80,0.55,0.70,NaN,...,0.40,NaN,0.25,0.2,0.35,NaN,0.75,NaN,NaN,NaN
3,NaN,NaN,0.55,0.55,0.20,0.85,0.55,0.50,NaN,0.55,...,0.50,0.30,NaN,0.2,0.45,NaN,NaN,NaN,0.60,0.80
4,NaN,0.35,0.35,NaN,0.30,0.25,0.90,0.45,NaN,0.40,...,0.50,NaN,NaN,0.5,0.35,0.50,0.95,0.60,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.95,0.25,0.40,0.25,0.30,NaN,0.45,NaN,NaN,NaN,...,0.65,NaN,NaN,0.4,NaN,0.85,0.90,0.55,0.80,NaN
6036,0.80,0.55,0.65,0.55,0.55,0.70,NaN,0.45,0.30,0.70,...,NaN,0.45,0.05,0.3,NaN,NaN,0.95,0.45,NaN,0.70
6037,NaN,0.55,0.55,NaN,0.65,NaN,NaN,0.50,NaN,NaN,...,0.30,0.30,0.30,NaN,0.65,0.60,NaN,0.50,0.65,NaN
6038,NaN,0.55,0.40,NaN,NaN,0.65,NaN,NaN,NaN,NaN,...,0.35,0.35,0.10,0.4,NaN,0.70,NaN,NaN,NaN,0.90


In [50]:
user_input_percent = 0.3
subset_percent = 0.7

num_ratings = len(ratings_imputed)
num_user_input = int(num_ratings * user_input_percent)
num_subset = int(num_ratings * subset_percent)

user_input_ratings = ratings_imputed.sample(n=num_user_input, replace=True, random_state=42)
subset_ratings = ratings_imputed.drop(user_input_ratings.index)
user_input_ratings

,user_id,movie_id,rating
6423388,2889,116,0.80
6550634,2946,882,0.95
10081351,4533,2931,0.90
4304572,1936,790,0.25
13315092,5988,1428,0.70
...,...,...,...
1675841,753,1667,0.65
11781798,5298,2507,0.45
5459974,2456,62,0.55
7858612,3534,1285,0.70


Imputation portion ends here

In [ ]:
# movies['title'] = movies['title'].str.replace(r'\(\d\d\d\d\)', '')
# movies['title'] = movies['title'].apply(lambda x: x.strip())
# movies.head()

In [ ]:
# userInput = [
#             {'title':'Breakfast Club, The', 'rating':5},
#             {'title':'Toy Story', 'rating':3.5},
#             {'title':'Jumanji', 'rating':2},
#             {'title':"Pulp Fiction", 'rating':5},
#             {'title':'Akira', 'rating':4.5}
#          ]
# inputMovies = pd.DataFrame(userInput)
# inputMovies

In [ ]:
# # inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]
# # inputMovies = pd.merge(inputId, inputMovies)
# inputMovies = inputMovies.drop('release_date', 1)
# inputMovies

In [51]:
userSubset = subset_ratings[subset_ratings['movie_id'].isin(user_input_ratings['movie_id'].tolist())]
userSubset.head()

,user_id,movie_id,rating
0,0,0,1.00
2,0,6,0.75
3,0,8,0.60
5,0,13,0.55
6,0,15,0.70


In [52]:
userSubsetGroup = userSubset.groupby(['user_id'])
userSubsetGroup.get_group(222)
mu_k=userSubsetGroup['rating'].mean()

In [53]:
userSubsetGroup=sorted(userSubsetGroup,key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

<ipython-input-53-5489e10f2460>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup=sorted(userSubsetGroup,key=lambda x: len(x[1]), reverse=True)


[(4712,
            user_id  movie_id  rating
  10477504     4712         2    0.50
  10477506     4712         5    1.00
  10477507     4712         6    0.65
  10477508     4712         7    0.55
  10477509     4712         8    0.40
  ...           ...       ...     ...
  10479805     4712      3680    0.65
  10479807     4712      3685    0.75
  10479808     4712      3686    0.40
  10479813     4712      3699    0.50
  10479814     4712      3705    0.75
  
  [1755 rows x 3 columns]),
 (4046,
           user_id  movie_id  rating
  8995778     4046         1    0.70
  8995779     4046         2    0.55
  8995781     4046         4    0.65
  8995782     4046         6    0.85
  8995783     4046         7    0.40
  ...          ...       ...     ...
  8998078     4046      3694    0.30
  8998079     4046      3695    0.40
  8998082     4046      3699    0.45
  8998085     4046      3703    0.70
  8998087     4046      3705    0.85
  
  [1749 rows x 3 columns]),
 (236,
          user_

In [54]:
userSubsetGroup=userSubsetGroup[0:500]

In [55]:
ratings_median=ratings['rating'].median()
print(ratings_median)

0.75


In [56]:
def agreement(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    above_median1 = rating1 > rating_median
    below_median1 = rating1 < rating_median
    above_median2 = rating2 > rating_median
    below_median2 = rating2 < rating_median

    agreement_condition1 = np.logical_and(above_median1, below_median2)
    agreement_condition2 = np.logical_and(below_median1, above_median2)

    return np.logical_or(agreement_condition1, agreement_condition2).any()

def proximity(rating1, rating2, rating_max, rating_min, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    agree = agreement(rating1, rating2, rating_median)
    dist = np.where(agree, np.absolute(rating1 - rating2), 2 * np.absolute(rating1 - rating2))
    prox = ((2 * (rating_max - rating_min) + 1) - dist) ** 2
    return prox

def impact(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    impact_score = (np.absolute(rating1 - rating_median) + 1) * (np.absolute(rating2 - rating_median) + 1)
    agree = agreement(rating1, rating2, rating_median)
    impact_result = np.where(agree, impact_score, 1 / impact_score)
    return impact_result

def popularity(rating1, rating2, mu_k):
    pop = np.where(
        (rating1 > mu_k) & (rating2 > mu_k) | (rating1 < mu_k) & (rating2 < mu_k),
        1 + ((rating1 + rating2) / 2 - mu_k) ** 2,
        1
    )
    return pop

In [57]:
import math
def agreement(rating1, rating2, rating_median):
    rating1 = np.asarray(rating1)
    rating2 = np.asarray(rating2)

    above_median1 = rating1 > rating_median
    below_median1 = rating1 < rating_median
    above_median2 = rating2 > rating_median
    below_median2 = rating2 < rating_median

    agreement_condition1 = np.logical_and(above_median1, below_median2)
    agreement_condition2 = np.logical_and(below_median1, above_median2)

    return np.logical_or(agreement_condition1, agreement_condition2).any()

def proximity(rating1, rating2, rating_max, rating_min, rating_median):
    D = np.abs(rating1 - rating2)
    med_plus = (rating_max + rating_median) / 2
    med_minus = (rating_min + rating_median) / 2
    Rmax_minus_Rmin_squared = (rating_max - rating_min) ** 2

    agree = agreement(rating1, rating2, rating_median)
    prox = (D - med_plus + med_minus) ** 2 / Rmax_minus_Rmin_squared
    delta = 0.75 if np.abs(rating1 - rating2) > rating_median else (0.5 if np.abs(rating1 - rating2) == rating_median else 0.25)
    return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))

def impact(rating1, rating2, rating_median):
    agree = agreement(rating1, rating2, rating_median)
    impact1= math.exp(-1 / ((np.abs(rating1 - rating_median) + 1) * (np.abs(rating2 - rating_median) + 1)))
    impact2= math.exp(1 / ((np.abs(rating1 - rating_median) + 1) * (np.abs(rating2 - rating_median) + 1)))
    return impact1 if agree else impact2

def proposed_popularity(rating1, rating2,rIi):
    if (rating1 > rIi and rating2 > rIi) or (rating1 < rIi and rating2 < rIi):
        return math.log10(2 + (((rating1 + rating2) / 2) - rIi) ** 2)
    else:
        return 0.3010


In [58]:
import numpy as np
import pandas as pd

def pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, rating_median, mu_k):
    pip_similarity_sum = 0
    n_ratings = 0

    for rating1, rating2 in zip(tempRatingList, tempGroupList):
        if not np.isnan(rating1) and not np.isnan(rating2):
            prox = proximity(rating1, rating2, rating_max, rating_min, rating_median)
            imp = impact(rating1, rating2, rating_median)
            pop = popularity(rating1, rating2,rating_median )

            pip_similarity_sum += prox * imp * pop
            n_ratings += 1

    if n_ratings == 0:
        return 0

    pip_similarity_value = pip_similarity_sum / n_ratings
    return pip_similarity_value

PIPdict = {}

rating_max = 5
rating_min = 0

for name, group in userSubsetGroup:

    group = group.sort_values(by='movie_id')
    inputMovies = user_input_ratings.sort_values(by='movie_id')
    temp_df = inputMovies[inputMovies['movie_id'].isin(group['movie_id'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    pip_similarity_value = pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, ratings_median, mu_k)
    PIPdict[name] = pip_similarity_value.mean()

print(sorted(PIPdict.keys()))

<ipython-input-57-d47755b63b66>:25: RuntimeWarning: divide by zero encountered in scalar divide
  return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))


[8, 20, 42, 52, 56, 62, 65, 67, 103, 113, 123, 133, 139, 146, 167, 187, 192, 202, 206, 220, 229, 234, 236, 262, 280, 281, 288, 318, 320, 343, 374, 377, 383, 404, 406, 417, 422, 435, 436, 479, 484, 489, 522, 524, 534, 541, 546, 551, 554, 563, 590, 594, 659, 668, 678, 682, 683, 700, 709, 732, 735, 756, 757, 762, 763, 771, 785, 799, 812, 813, 823, 828, 829, 850, 858, 878, 887, 892, 894, 895, 896, 906, 921, 932, 965, 978, 993, 1003, 1009, 1013, 1024, 1025, 1031, 1037, 1039, 1058, 1069, 1075, 1086, 1092, 1099, 1129, 1143, 1147, 1160, 1162, 1175, 1191, 1195, 1234, 1235, 1239, 1242, 1270, 1290, 1299, 1302, 1316, 1341, 1350, 1356, 1381, 1395, 1400, 1431, 1458, 1471, 1507, 1509, 1536, 1539, 1559, 1567, 1584, 1598, 1615, 1628, 1636, 1647, 1651, 1658, 1666, 1667, 1671, 1672, 1673, 1674, 1686, 1687, 1693, 1709, 1729, 1734, 1759, 1801, 1828, 1834, 1844, 1916, 1926, 1941, 1942, 1943, 1945, 1947, 1970, 1972, 1973, 2007, 2016, 2031, 2046, 2065, 2067, 2072, 2075, 2084, 2089, 2103, 2128, 2186, 2191, 219

In [60]:
print(PIPdict[8])

inf


In [61]:
import numpy as np
import pandas as pd

def pip_similarity(tempRatingList, tempGroupList, rating_max, rating_min, rating_median, mu_k):
    pip_similarity_sum = 0
    n_ratings = 0

    for rating1, rating2 in zip(tempRatingList, tempGroupList):
        if not np.isnan(rating1) and not np.isnan(rating2):
            prox = proximity(rating1, rating2, rating_max, rating_min, rating_median)
            imp = impact(rating1, rating2, rating_median)
            pop = popularity(rating1, rating2,rating_median )

            pip_similarity_sum += prox * imp * pop
            n_ratings += 1

    if n_ratings == 0:
        return 0

    pip_similarity_value = pip_similarity_sum / n_ratings
    return pip_similarity_value

PIPdict = {}

rating_max = 5
rating_min = 0

for name, group in userSubsetGroup:
  group = group.sort_values(by='movie_id')
  inputMovies = user_input_ratings.sort_values(by='movie_id')
  temp_df = inputMovies[inputMovies['movie_id'].isin(group['movie_id'].tolist())]
  tempRatingList = temp_df['rating'].tolist()
  tempGroupList = group['rating'].tolist()

  similarity_values = []

  for temp_rating, group_rating in zip(tempRatingList, tempGroupList):
      pip_similarity_value = pip_similarity([temp_rating], [group_rating], rating_max, rating_min, ratings_median, mu_k)
      similarity_values.append(pip_similarity_value)

print(similarity_values)

<ipython-input-57-d47755b63b66>:25: RuntimeWarning: divide by zero encountered in scalar divide
  return prox if agree else (delta * 1 / (D *  Rmax_minus_Rmin_squared))


[0.23119878156522705, 0.059928246699386394, 0.09156866178530144, 0.09293582903639577, 0.08797933608246876, 0.4254385256591562, 0.08358158041774556, 0.11504879454464127, 0.08296006648372738, 0.15905821945132698, inf, 0.08902163713969871, 0.47805580431849237, 0.15905821945132698, 0.07193685113161484, 0.10952779758546313, 0.24820650846230127, 0.14973395081548393, 0.11504879454464127, 0.08887353514489933, 0.09075997797754956, inf, 0.09185635999623888, 0.059928246699386394, 0.08902163713969871, 0.44779493223641975, 0.09243820341661502, 0.08902163713969871, inf, 0.08765892610857924, 0.09156866178530144, 0.09243820341661502, 0.09036792684776553, 0.08436678784688777, 0.08902163713969871, 0.03113743262387037, 0.09156866178530144, 0.4254385256591562, 0.07186416188997889, 0.09156866178530144, 0.08797933608246876, 0.10952779758546313, inf, 0.09156866178530144, 0.09036792684776553, 0.44779493223641975, 0.0847408272885694, 0.14973395081548393, 0.07186416188997889, 0.5183746891629745, 0.0718641618899

In [62]:
#PIP_DF = pd.DataFrame.from_dict(PIPdict, orient='index')
PIP_DF = pd.DataFrame(similarity_values, columns=['Similarity'])
PIP_DF.columns = ['similarityIndex']
PIP_DF['userId'] = PIP_DF.index
PIP_DF.index = range(len(PIP_DF))

#removing the inf values

PIP_DF['similarityIndex'] = PIP_DF['similarityIndex'].replace([np.inf, -np.inf], 0)
PIP_DF.head()

,similarityIndex,userId
0,0.231199,0
1,0.059928,1
2,0.091569,2
3,0.092936,3
4,0.087979,4


In [63]:
topUsers=PIP_DF.sort_values(by='similarityIndex', ascending=False)
topUsers.head(100)

,similarityIndex,userId
1370,1.295992,1370
1432,1.167822,1432
1420,0.908797,1420
844,0.518375,844
1149,0.518375,1149
...,...,...
12,0.478056,12
798,0.478056,798
1414,0.478056,1414
367,0.478056,367


In [64]:
topUsersRating=topUsers.merge(ratings, left_on='userId', right_on='user_id', how='inner')
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
#topUsersRating.head(18000)
topUsersRating.head(1352)
topUsersRating.isnull().sum()

similarityIndex    0
userId             0
user_id            0
movie_id           0
rating             0
weightedRating     0
dtype: int64

In [65]:
tempTopUsersRating = topUsersRating.groupby('movie_id').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
#handled the zero division error bug
recommendation_df.loc[tempTopUsersRating['sum_similarityIndex'] == 0, 'weighted average recommendation score'] = 0
recommendation_df.fillna(0, inplace=True)
##
recommendation_df['movieID'] = tempTopUsersRating.index
recommendation_df.tail()

,weighted average recommendation score,movieID
movie_id,,
3948,0.670757,3948
3949,0.815004,3949
3950,0.545948,3950
3951,0.702948,3951
3952,0.681843,3952


In [66]:
tempTopUsersRating.head(973)

,sum_similarityIndex,sum_weightedRating
movie_id,,
1,83.801423,65.272883
2,31.244927,17.215038
3,22.329543,11.640159
4,9.421295,3.609010
5,14.585635,6.826877
...,...,...
1089,51.267505,40.193478
1090,47.205271,36.171377
1091,18.696539,7.535573


In [67]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(100)

,weighted average recommendation score,movieID
movie_id,,
3314,1.000000,3314
3280,1.000000,3280
787,1.000000,787
3800,1.000000,3800
3205,1.000000,3205
...,...,...
1204,0.843166,1204
1267,0.843033,1267
1361,0.842984,1361


In [68]:
recommendation_df = recommendation_df[(recommendation_df['weighted average recommendation score'] <= 5) &
 (recommendation_df['weighted average recommendation score'] >= 0) &
 (recommendation_df['weighted average recommendation score'].notnull())]
print(recommendation_df)

          weighted average recommendation score  movieID
movie_id                                                
3314                                        1.0     3314
3280                                        1.0     3280
787                                         1.0      787
3800                                        1.0     3800
3205                                        1.0     3205
...                                         ...      ...
730                                         0.0      730
1165                                        0.0     1165
3828                                        0.0     3828
3621                                        0.0     3621
3294                                        0.0     3294

[3555 rows x 2 columns]


In [69]:
print(recommendation_df.isnull().sum())

weighted average recommendation score    0
movieID                                  0
dtype: int64


In [70]:
recommendation_df.head(10)

,weighted average recommendation score,movieID
movie_id,,
3314,1.0,3314
3280,1.0,3280
787,1.0,787
3800,1.0,3800
3205,1.0,3205
854,1.0,854
578,1.0,578
3486,1.0,3486
2984,1.0,2984


In [71]:
movies.loc[movies['movie_id'].isin(recommendation_df.head(10)['movieID'].tolist())]

,movie_id,title
574,578,"Hour of the Pig, The (1993)"
594,598,Window to Paris (1994)
777,787,"Gate of Heavenly Peace, The (1995)"
843,854,"Ballad of Narayama, The (Narayama Bushiko) (1958)"
2915,2984,On Any Sunday (1971)
3136,3205,Black Sunday (La Maschera Del Demonio) (1960)
3211,3280,"Baby, The (1973)"
3245,3314,"Big Trees, The (1952)"
3417,3486,Devil Girl From Mars (1954)
3731,3800,Criminal Lovers (Les Amants Criminels) (1999)


In [72]:
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [73]:
from math import sqrt
merged_ratings = pd.merge(recommendation_df, ratings,on='movie_id')
merged_ratings['squared_diff'] = (merged_ratings['weighted average recommendation score'] - merged_ratings['rating']) ** 2
mean_squared_diff = merged_ratings['squared_diff'].mean()
mean_squared_diff
rmse_score = sqrt(mean_squared_diff)
print("RMSE Score:", rmse_score)

RMSE Score: 0.24640206286417632


In [74]:
import pandas as pd
from math import sqrt
merged_ratings = pd.merge(recommendation_df, ratings, on='movie_id')

merged_ratings['abs_diff'] = abs(merged_ratings['weighted average recommendation score'] - merged_ratings['rating'])

mae_score = merged_ratings['abs_diff'].mean()

print("MAE Score:", mae_score)

MAE Score: 0.1965232707376145
